In [1]:
import json
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, lag, stddev, avg, count
from snowflake.snowpark.window import Window


In [2]:
with open('../config.json', 'r') as f:
    config = json.load(f)

In [3]:
# Create Snowpark session
session = Session.builder.configs({
    "account": config["account"],
    "user": config["user"],
    "password": config["password"],
    "database": config["database"],
    "schema": config["schema"]
}).create()

In [ ]:
df = session.table("player_yearly_performance")
window = Window.partition_by(col("player_name")).order_by(col("year"))
df_trends = df.with_column("prev_win_rate", lag(col("wins") / col("matches"), 1).over(window))
df_trends = df_trends.with_column("win_rate_change", (col("wins") / col("matches") - col("prev_win_rate")))

df_trends.write.mode("overwrite").save_as_table("player_performance_trends")

In [ ]:
# Serve Effectiveness by Surface
df_matches = session.table("player_matches_view").join(
    session.table("matches").select(col("TOURNEY_DATE"), col("SURFACE"), col("WINNER_ID"), col("LOSER_ID")),
    (col("PLAYER_ID") == col("WINNER_ID")) | 
    (col("PLAYER_ID") == col("LOSER_ID"))
)
df_serve = df_matches.group_by(["PLAYER_NAME", "SURFACE"]).agg(
    avg("FIRST_SERVE_PCT").alias("avg_first_serve_pct"),
    stddev("FIRST_SERVE_PCT").alias("stddev_first_serve_pct")
).filter(col("avg_first_serve_pct").is_not_null())
df_serve.write.mode("overwrite").save_as_table("serve_effectiveness_by_surface")

In [ ]:
# Cell 5: Player Consistency Across Years
df = session.table("player_yearly_performance")
# Calculate win rate per year
df_with_win_rate = df.with_column("win_rate", col("wins") / col("matches"))
# Group by player to compute standard deviation of win rate
df_consistency = df_with_win_rate.group_by(["player_name"]).agg(
    avg("win_rate").alias("avg_win_rate"),
    stddev("win_rate").alias("win_rate_stddev"),
    count("*").alias("years_active")
).filter(col("years_active") > 5)  # Filter players with significant activity
df_consistency.write.mode("overwrite").save_as_table("player_consistency")

In [ ]:
session.close()